<a href="https://colab.research.google.com/github/TOKTAK007/DL_23/blob/master/Alexnet_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#-----------------------------------------------------------------------------------------#
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm_notebook
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim 
import torchvision
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms
import torch.nn.functional as F
#-----------------------------------------------------------------------------------------#
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
import time
#-----------------------------------------------------------------------------------------#
from torch.utils.data import random_split, DataLoader
from torchvision import transforms
from torchsummary import summary

====================================

โค้ดดังกล่าวเป็นการ import ฟังก์ชัน tqdm_notebook จากโมดูล tqdm

tqdm เป็นโมดูลที่ช่วยในการแสดง progress bar ของ loop ในการประมวลผลข้อมูล โดย tqdm_notebook เป็นฟังก์ชันที่ใช้กับ Jupyter Notebook เพื่อแสดง progress bar ใน cell ของ Jupyter Notebook โดยเฉพาะ

การใช้ tqdm_notebook จะช่วยให้ผู้ใช้งานสามารถติดตามการดำเนินการของโปรแกรมได้ง่ายขึ้น และทำให้การแก้ไขโค้ดหรือการปรับปรุงโมเดลสามารถดูผลอย่างรวดเร็วได้ง่ายขึ้น

---



โค้ดดังกล่าวเป็นการกำหนดค่าต่างๆ สำหรับการทำงานของโมเดล Machine Learning โดยมีค่าต่างๆ ดังนี้

DATASET_PATH คือ path ที่เก็บข้อมูล (datasets) ซึ่งอยู่ในโฟลเดอร์ '../larger_than_50_MB/team_datasets/Aum/datasets'
train_size คือขนาดของชุดข้อมูลสำหรับ train model ซึ่งเท่ากับ 70% ของข้อมูลทั้งหมด
val_size คือขนาดของชุดข้อมูลสำหรับ validation model ซึ่งเท่ากับ 20% ของข้อมูลทั้งหมด
test_size คือขนาดของชุดข้อมูลสำหรับทดสอบ model ซึ่งเท่ากับ 10% ของข้อมูลทั้งหมด
seed คือค่าเริ่มต้นของ random seed ที่ใช้ในการสุ่มข้อมูล
num_workers คือจำนวน worker ที่ใช้ในการประมวลผลข้อมูลในช่วงการ training โดยใช้ CPU ซึ่งมีจำนวน 24 ตัว
batch_size คือจำนวนข้อมูล (ตัวอย่าง) ที่ใช้ในแต่ละรอบการสร้างโมเดล
model_name คือชื่อของโมเดลที่ใช้ในการสร้าง (Aum)

=======================================



---



os เป็นโมดูลของ Python ที่ใช้สำหรับจัดการระบบปฏิบัติการ (Operating System) ในการเข้าถึงไฟล์ โฟลเดอร์ การทำงานกับ process การกำหนด permissions และการจัดการกับ environment variables ฯลฯ

os ช่วยในการเข้าถึงระบบได้อย่างมีประสิทธิภาพและแม่นยำ โดยไม่จำเป็นต้องเขียนโค้ดให้ซับซ้อนเอง นอกจากนี้ os ยังช่วยในการจัดการ path ของไฟล์และโฟลเดอร์ในการเข้าถึงไฟล์อย่างสะดวกและเข้าใจง่าย


---



In [ ]:
#Step 0: Predefined Parameters.

In [ ]:
DATASET_PATH = '../larger_than_50_MB/team_datasets/Aum/datasets'
train_size = 0.7; val_size = 0.2; test_size = 0.1
seed = 53
num_workers = 24 # CPU
batch_size = 32
model_name = 'Aum'



---
โค้ดดังกล่าวเป็นการกำหนด path ของไฟล์ checkpoint ที่จะเก็บโมเดลที่ train ไว้ โดยใช้ตัวแปร CHECKPOINT_PATH เพื่อเก็บ path ของไฟล์ checkpoint

โดยจะเป็นการเชื่อมต่อ path ของไดเรกทอรี '../larger_than_50_MB/save_trained_model/' กับชื่อโมเดล model_name และนามสกุลไฟล์ '.ckpt' เข้าด้วยกัน ดังนั้นจะได้ path สมบูรณ์ของไฟล์ checkpoint ที่เก็บโมเดลที่ train ไว้ในตำแหน่งที่กำหนดขึ้นมา


---






---
.ckpt คือนามสกุลของไฟล์ checkpoint ใน PyTorch Lightning ซึ่งใช้สำหรับเก็บโมเดลที่ train ไว้ เมื่อเรา train โมเดลด้วย PyTorch Lightning จะสร้างไฟล์ .ckpt ไว้ทุกครั้งที่เกิดการเซฟโมเดล เพื่อใช้ในการเรียกโมเดลกลับมาใช้งานต่อไป นอกจากนี้ .ckpt ยังเป็นไฟล์ที่ใช้เก็บสถานะ (state) ของโมเดล รวมถึงค่า weight, bias, optimizer state, epoch, step และอื่นๆ ซึ่งทำให้เราสามารถเรียกโมเดลกลับมา train ต่อได้โดยไม่ต้องเริ่มต้นการ train ใหม่ทั้งหมด


---


